<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analysis-of-holdings" data-toc-modified-id="Analysis-of-holdings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analysis of holdings</a></span><ul class="toc-item"><li><span><a href="#Most-common-stocks" data-toc-modified-id="Most-common-stocks-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Most common stocks</a></span></li><li><span><a href="#Merge-Stock_map-and-Hold_co_info-to-get-name-of-each-security" data-toc-modified-id="Merge-Stock_map-and-Hold_co_info-to-get-name-of-each-security-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Merge Stock_map and Hold_co_info to get name of each security</a></span></li><li><span><a href="#Get-stocks-for-fund-by-port_no" data-toc-modified-id="Get-stocks-for-fund-by-port_no-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Get stocks for fund by port_no</a></span></li><li><span><a href="#Distribution-by-date" data-toc-modified-id="Distribution-by-date-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Distribution by date</a></span></li><li><span><a href="#Observations-per-Fund" data-toc-modified-id="Observations-per-Fund-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Observations per Fund</a></span></li><li><span><a href="#Map-Obj-codes-to-returns" data-toc-modified-id="Map-Obj-codes-to-returns-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Map Obj codes to returns</a></span></li><li><span><a href="#Aggregate-return" data-toc-modified-id="Aggregate-return-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Aggregate return</a></span></li><li><span><a href="#Objective-codes" data-toc-modified-id="Objective-codes-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Objective codes</a></span></li><li><span><a href="#Precprocessing-holdings-data" data-toc-modified-id="Precprocessing-holdings-data-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Precprocessing holdings data</a></span></li></ul></li></ul></div>

# Analysis of holdings

Description:



In [ ]:
import feather
import numpy as np
import pandas as pd

from scipy import sparse

import matplotlib.pyplot as plt

In [ ]:
path = '../data/raw/holdings_co_info.feather'
holdings_co_info = feather.read_dataframe(path)
holdings_co_info = holdings_co_info.drop_duplicates()
holdings_co_info.shape

In [ ]:
path = '../data/processed/holdings_summary_s_s.feather'
summary = feather.read_dataframe(path)
summary.shape

In [ ]:
path = '../data/processed/stock_map.feather'
stock_map = feather.read_dataframe(path)
stock_map.shape

In [ ]:
path = '../data/processed/holdings_b_b.npz'
holdings = sparse.load_npz(path)
holdings.shape

## Most common stocks

In [ ]:
holdings_summary = pd.DataFrame(holdings.sum(0).T)
holdings_summary.columns = ['total']
holdings_summary.shape

In [ ]:
holdings_df = pd.DataFrame(holdings.toarray())
holdings_df = holdings_df.groupby(by = summary['port_no']).tail(5)

In [ ]:
holdings_df.shape

## Merge Stock_map and Hold_co_info to get name of each security

In [ ]:
stock_map.columns = ['crsp_company_key','nr','1']
stock_map = stock_map.drop(columns = '1')

In [ ]:
holdings_co_info.sample()

In [ ]:
stock_map_merged = stock_map.merge(holdings_co_info, how='left', left_on = 'crsp_company_key', right_on='crsp_company_key')

In [ ]:
stock_map_merged = pd.concat([holdings_summary,stock_map_merged],axis=1)

In [ ]:
stock_map_merged.loc[stock_map_merged['security_name'].astype('str').str.contains('APPLE')]

In [ ]:
stock_map_merged.sort_values(by=['total'],ascending=False).head(100)

In [ ]:
summary[summary['port_no'] == 1026915]

## Get stocks for fund by port_no

TODO Kann irgendwie nicht sein 

In [ ]:
stocks_nr = pd.DataFrame({
 'nr' : holdings[35852].nonzero()[1]   
})

In [ ]:
stock_map_merged = stock_map.merge(holdings_co_info, how='left', left_on = 'crsp_company_key', right_on='crsp_company_key')

In [ ]:
(stocks_nr
     .merge(stock_map_merged, how='left', left_on = 'nr', right_on='nr')
     .dropna()
     .head()
)

## Distribution by date

In [ ]:
# Analysis of selected funds

## Distribution by date(returns['report_dt']
(returns['crsp_fundno']
    .groupby([
         returns['caldt'].dt.year,
#         returns['caldt'].dt.month
     ])
     .count()
     .plot(kind='bar',
           color = 'b',
          figsize=(20,10)))

## Observations per Fund

In [ ]:
# Analysis of selected funds

## Distribution by date(returns['report_dt']
a = (returns['crsp_fundno']
    .groupby([
         returns['crsp_fundno']
     ])
    .count()
)
a = a.groupby(a).count().cumsum().T

In [ ]:
a.plot(kind='line',
       color = 'b',
       figsize=(20,10))

In [ ]:
(returns.loc[returns['crsp_fundno'] == 27650,'mret']
    .plot(kind='bar',
          color = 'b',
          figsize=(20,10)))

## Map Obj codes to returns

In [ ]:
summary.head().dtypes

In [ ]:
returns.head().dtypes

In [ ]:
returns_merged = returns.merge(summary, how='left', on=['crsp_fundno'])
returns_merged = returns_merged.dropna(axis = 0, how = 'any')

In [ ]:
returns_merged.sample(10)

## Aggregate return

In [ ]:
returns_merged.sample(5)

In [ ]:
# Analysis of selected funds
return_class = returns_merged[returns_merged['crsp_obj_cd'] == 'EFSH']

In [ ]:
return_class = (return_class['mret']
    .groupby([
        return_class['crsp_obj_cd'],
        return_class['caldt']
     ])
    .agg(['mean','count']))

In [ ]:
return_class.loc[:,'cumulative'] = (return_class.loc[:,'mean'].fillna(0)+1).cumprod()

Cum return over time

In [ ]:
return_class['mean'].plot(kind='line',
           color = 'b',
          figsize=(16,10))

Different from zero?

In [ ]:
return_class.loc[:,['mean']].boxplot()

## Objective codes

In [ ]:
returns_grouped = (returns_merged['cum']
    .groupby([
        returns_merged['crsp_obj_cd'],
#        returns_merged['caldt']
     ]))

In [ ]:
returns_grouped = returns_grouped.describe()

In [ ]:
returns_grouped['mean'].sort_values().plot(
            kind='bar',
            color = 'b',
            figsize=(16,10))

## Precprocessing holdings data


In [ ]:
holdings_svd = np.array(svd.fit_transform(holdings_f))
holdings_svd

In [ ]:
holdings_scaled = scale(holdings_f,with_mean=False)
holdings_scaled

In [ ]:
holdings_maxabs = MaxAbsScaler().fit_transform(holdings_f)
holdings_maxabs

In [ ]:
plt.figure(figsize=(20,8))

g = sns.distplot(np.array(holdings_f.data).flatten(),label='Original',bins=1000,kde = False)
sns.distplot(holdings_svd.flatten(),label='SVD',bins=1000, kde = False)
#sns.distplot(np.array(holdings_scaled.data).flatten(),label='Scaled',bins=1000,kde = False)
g = sns.distplot(np.array(holdings_maxabs.data).flatten(),label='MaxAbs',bins=1000,kde = False)

plt.legend()
g.set(xlim=[0,1])